In [1]:
import sys
sys.path.append('/home/project-1/baoziyuan/ChatGLM-RPC-Tutorial/')
from transformers import AutoTokenizer, AutoModel, AutoConfig
import json
import os
import pickle
from asyncio import sleep
from queue import Queue

import torch
import torch.nn as nn
from tqdm import tqdm
from functools import partial

from chatglm_6b_split_server import ChatGLMTokenizer, ChatGLMConfig, ChatGLMForConditionalGeneration
import os
from IPython.display import display, Markdown, clear_output
from importlib import reload


tokenizer_config_path = os.path.join("..","..","..","checkpoints","chatglm-6b", 'tokenizer_config.json')
model_config_path = os.path.join("..","..","..","checkpoints","chatglm-6b", 'config.json')
token_text_path = os.path.join("..","..","..","checkpoints","chatglm-6b", "ice_text.model")
model_dir = os.path.join("..","..","..","checkpoints","chatglm-6b")
checkpoint_path = os.path.join("output","adgen-chatglm-6b-pt-128-2e-2","checkpoint-3000")
model_state_dict_file_num = 8

def load_model(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
    config = AutoConfig.from_pretrained(model_dir, trust_remote_code=True, pre_seq_len=128)
    model = AutoModel.from_pretrained(model_dir, config=config, trust_remote_code=True).half().cuda()
    model = model.eval()
    return model, tokenizer

/home/project-1/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-19 08:59:44,941] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model, tokenizer = load_model(model_dir)
print(model)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:22<00:00,  2.86s/it]
Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at ../../../checkpoints/chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (prefix_encoder): PrefixEncoder(
      (embedding): Embedding(128, 229376)
    )
    (dropout): Dropout(p=0.1, inplace=Fal

In [5]:
import utility
#reload(utility)
model.eval()
hook_function = partial(utility.save_input_output, model)
handle = model.transformer.layers[6].register_forward_hook(hook_function)
#for name, param in model.named_parameters():
#    param.requires_grad = True
freeze_schedule = {5: [10, 11, 12], 
                10: [13, 14, 15]}
epoch = 10
utility.dynamic_freeze(model, freeze_schedule, epoch)



In [4]:
prefix_state_dict = torch.load(os.path.join(checkpoint_path, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

#V100 机型上可以不进行量化
#print(f"Quantized to 4 bit")
#model = model.quantize(4)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'output/adgen-chatglm-6b-pt-128-2e-2/checkpoint-3000/pytorch_model.bin'

In [ ]:
response, history = model.chat(tokenizer, "类型#上衣\*材质#牛仔布\*颜色#白色\*风格#简约\*图案#刺绣\*衣样式#外套\*衣款式#破洞", history=[])
print(response)

In [8]:
reload(utility)

<module 'utility' from '/home/project-1/baoziyuan/ChatGLM-RPC-Tutorial/2_Experiment/utility.py'>

In [6]:
prompt = "如何制作宫保鸡丁"
for response, history in model.stream_chat(
        tokenizer, prompt, history=[]):
    clear_output(wait=True)
    display(Markdown(response))

宫保鸡丁是一道非常有名的川菜，以下是制作宫保鸡丁的步骤：

材料：

- 鸡胸肉 1块
- 青椒 1个
- 干辣椒 10个
- 姜 适量
- 蒜 适量
- 料酒 适量
- 生抽 适量
- 盐 适量
- 糖 适量
- 鸡精 适量
- 淀粉 适量
- 食用油 适量

步骤：

1. 将鸡胸肉切成丁，加入料酒、淀粉腌制15分钟。
2. 干辣椒剪成段，姜、蒜切末备用。
3. 青椒去籽，切成小块备用。
4. 热锅凉油，将鸡丁煸炒至变色，加入干辣椒、姜蒜末继续煸炒。
5. 加入适量的生抽、盐、糖、鸡精，翻炒均匀。
6. 最后加入青椒块，继续翻炒几分钟即可。

这是一道非常经典的川菜，希望您能喜欢。

: 

In [9]:
input_tensor = model.transformer.layers[0].input_tensor
output_tensor = model.transformer.layers[0].output_tensor
print(output_tensor.shape)

torch.Size([1, 1, 4096])


: 